In [1]:
import sys
import os
import time
import subprocess
from pyhocon import ConfigFactory


# Configure python path to load modules
# Get the current working directory. Should be 'services'
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'services', 'Current directory is not services'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the 'software' folder. Otherwise, something went wrong.
assert os.path.exists(os.path.join(parent_dir, 'software')), 'software folder not found in the repository root'

stm_dt_software_dir = os.path.join(parent_dir, 'software')

assert os.path.exists(stm_dt_software_dir), 'stm_dt/software directory not found'

# Add the parent directory to sys.path
sys.path.append(stm_dt_software_dir)

# Import RPCClient class from STM DT, which makes connecting to RabbitMQ and calling remote methods easier.
from communication.rpc_client import RPCClient


In [2]:
# Start the process asynchronously
service_proc = subprocess.Popen([sys.executable, "gps_telemetry_validation_service.py"])

# Wait for 5 seconds for the process to start
time.sleep(5)

# Print the PID of the process.
print(service_proc.pid)


2025-03-19 20:34:38.968 INFO MongoDb : log6953fe_db database exists.
2025-03-19 20:34:38.968 INFO MongoDb : bus_stops collection exists in database log6953fe_db.
2025-03-19 20:34:38.981 INFO GPSCoordinatesAnomalyService : GPSCoordinatesAnomalyService setup complete.
24561


In [3]:
# Print contents of log file to verify that the service is running

with open('logs/dt.log', 'w+') as f:
    print(f.read())

# Check process has not crashed
assert service_proc.poll() is None, "GPSCoordinatesAnomalyService process has crashed"


In [ ]:
# Get config
import time
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'software', 'config', 'startup.conf')
config = ConfigFactory.parse_file(startup_conf)

with RPCClient(**(config["rabbitmq"])) as client:
    data = {
        "source": "stm_api_bus_update",
        "time": time.time_ns(),
        "tags": {
            "source": "stm_api_bus_update"
        },
        'data': [
            {
                'route_id': 121,
                'stop_sequence': 10,
                'latitude': 45.565391,
                'longitude': -73.646170
            }
        ]
    }
    
    reply = client.invoke_method("gps.coordinates.validation.service", "validate_gps_coordinates", {"payload": data})
    print(reply)

# Check process has not crashed
assert service_proc.poll() is None, "GPSCoordinatesValidationService process has crashed"
